In [7]:
import pandas as pd

# Load weather data
weather_df = pd.read_csv("GlobalWeatherRepository.csv")
weather_df.columns = weather_df.columns.str.strip()

# Extract date
weather_df["Date"] = pd.to_datetime(weather_df["last_updated"], errors="coerce").dt.date.astype(str)

# Rename and keep relevant features
weather_df = weather_df.rename(columns={
    "location_name": "City",
    "temperature_celsius": "Temp",
    "wind_kph": "Wind",
    "condition_text": "Condition"
})
weather_df = weather_df[["City", "Date", "Temp", "Wind", "Condition"]]
weather_df = weather_df.dropna(subset=["Temp", "Wind"])
weather_df["City"] = weather_df["City"].str.strip().str.title()


In [9]:
def launch_score(temp, wind, condition):
    score = 100
    if temp < 0 or temp > 35:
        score -= 20
    if wind > 30:
        score -= 30
    if isinstance(condition, str):
        condition = condition.lower()
        if "rain" in condition or "storm" in condition:
            score -= 25
        elif "cloud" in condition:
            score -= 10
    return max(score, 0)

weather_df["Score"] = weather_df.apply(lambda row: launch_score(row["Temp"], row["Wind"], row["Condition"]), axis=1)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Extract month
weather_df["Month"] = pd.to_datetime(weather_df["Date"]).dt.month

# Encode city
le_city = LabelEncoder()
weather_df["City_enc"] = le_city.fit_transform(weather_df["City"])

# Final dataset
X = weather_df[["City_enc", "Month", "Temp", "Wind"]]
y = weather_df["Score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📉 MSE: {mse:.2f}")
print(f"📈 R² Score: {r2:.2f}")


📉 MSE: 56.85
📈 R² Score: 0.56


# Prepare data

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Extract month
weather_df["Month"] = pd.to_datetime(weather_df["Date"], errors="coerce").dt.month

# Encode city
le_city = LabelEncoder()
weather_df["City_enc"] = le_city.fit_transform(weather_df["City"].str.strip().str.title())

# Drop missing
weather_df = weather_df.dropna(subset=["Temp", "Wind", "Month", "City_enc"])

# Features for weather prediction (Model 1)
X_weather = weather_df[["City_enc", "Month"]]
y_weather = weather_df[["Temp", "Wind"]]

# Features for score prediction (Model 2)
X_score = weather_df[["City_enc", "Month", "Temp", "Wind"]]
y_score = weather_df["Score"]


# Weather predictor

In [29]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

# Train/test split for weather model
Xw_train, Xw_test, yw_train, yw_test = train_test_split(X_weather, y_weather, test_size=0.2, random_state=42)

# Train weather predictor
weather_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
weather_model.fit(Xw_train, yw_train)


MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

# Score predictor

In [31]:
# Train/test split for score model
Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_score, y_score, test_size=0.2, random_state=42)

# Train score predictor
score_model = RandomForestRegressor(n_estimators=100, random_state=42)
score_model.fit(Xs_train, ys_train)


RandomForestRegressor(random_state=42)

In [62]:
from datetime import datetime

# === User Input ===
city_input = "Astana"
launch_date = "2027-09-14"

# Process input
city_input = city_input.strip().title()
if city_input not in le_city.classes_:
    print("❌ City not recognized.")
else:
    city_enc = le_city.transform([city_input])[0]
    month = datetime.strptime(launch_date, "%Y-%m-%d").month

    # Predict weather
    weather_input = pd.DataFrame([{
        "City_enc": city_enc,
        "Month": month
    }])
    temp_pred, wind_pred = weather_model.predict(weather_input)[0]

    # Predict score
    score_input = pd.DataFrame([{
        "City_enc": city_enc,
        "Month": month,
        "Temp": temp_pred,
        "Wind": wind_pred
    }])
    score_pred = score_model.predict(score_input)[0]

    print(f"📍 City: {city_input}, Month: {month}")
    print(f"🌡️ Predicted Temp: {round(temp_pred, 2)}°C")
    print(f"💨 Predicted Wind: {round(wind_pred, 2)} km/h")


📍 City: Astana, Month: 9
🌡️ Predicted Temp: 16.06°C
💨 Predicted Wind: 19.12 km/h
